# Train NanoDet with custom dataset
<a target="_blank" href="https://colab.research.google.com/github/SonySemiconductorSolutions/aitrios-rpi-tutorials-ai-model-training/blob/main/notebooks/nanodet-ppe/custom_nanodet.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Training NanoDet model to detect Personal Protection Equipment (PPE) using open source dataset.

Observe, that this tutorial requires GPU when used with Colab. Enable GPU:

* Navigate to Edit→Notebook Settings
* Select T4 GPU from Hardware Accelerator

Nanodet training based on https://github.com/RangiLyu/nanodet/tree/main

Tutorial includes:
- Dataset setup
- Nanodet model setup
- Training
- Quantization using [Model Compression Toolkit - MCT](https://github.com/sony/model_optimization)
- COCO evaluation
- Visualization
- Conversion

In [1]:
"""
Expected error message:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.15.1 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.5 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 3.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.15.1 which is incompatible.
jax 0.5.2 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.3.2 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.15.1 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
"""
!pip install --no-cache-dir torch~=2.5.0 torchvision tensorflow==2.15.1 pycocotools imx500-converter[tf] edge-mdt-tpc "numpy<2" opencv-python

ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.1 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.1


In [2]:
# Converter requires java
import os
import re
import subprocess

def install_java(package: str = 'openjdk-17-jdk', version: int = 17) -> bool:
    try:
        result = subprocess.run(['java', '--version'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        version_output = result.stdout.splitlines()[0]
        match = re.search(r'(\d+)\.(\d+)\.(\d+)', version_output)  # Match version in form major.minor.patch
        print(f"Found Java version: {match.group(0)}")
        if match:
            major_version = int(match.group(1))
            if major_version == version:
                return True
            else:
                print(f"Java {version} is not installed. Installing correct version...")
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print(f"Java not installed. Installing...")

    try:
        is_root = os.geteuid() == 0
        prefix = [] if is_root else ['sudo']
        with open(os.devnull, 'w') as devnull:
            subprocess.run(prefix + ['apt', 'install', '-y', package], check=True, stdout=devnull, stderr=devnull)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Installation error: {e}")
        return False

if install_java():
    print(f'Java installed')
else:
    print(f'Java missing and installation failed')

Found Java version: 11.0.28
Java 17 is not installed. Installing correct version...
Java installed


# Installation

In [3]:
"""
Known errors:
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
"""
NANODET_COMMIT = 'pytorch2.0'
!rm -rf nanodet
!git clone https://github.com/RangiLyu/nanodet.git
!touch nanodet/nanodet/model/__init__.py
!cd nanodet && git checkout {NANODET_COMMIT} && pip install -q --no-cache-dir -r requirements.txt

Cloning into 'nanodet'...
remote: Enumerating objects: 2722, done.
remote: Total 2722 (delta 0), reused 0 (delta 0), pack-reused 2722 (from 1)
Receiving objects: 100% (2722/2722), 5.29 MiB | 4.63 MiB/s, done.
Resolving deltas: 100% (1602/1602), done.
Branch 'pytorch2.0' set up to track remote branch 'pytorch2.0' from 'origin'.
Switched to a new branch 'pytorch2.0'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 64.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 54.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 96.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 86.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

In [8]:
# Perform initial checks in order to continue
import os
import shutil
import tensorflow as tf
import torch
import numpy as np

#assert '2.15' in tf.__version__, print(tf.__version__)
#assert '2.5.' in torch.__version__, print(torch.__version__)
#assert '1' in np.__version__[0], print(np.__version__)

print(f'Is cuda available: {torch.cuda.is_available()}')
assert torch.cuda.is_available(), "GPU is required, for Colab see for example, https://colab.research.google.com/notebooks/gpu.ipynb" # training requires 254M

Is cuda available: True


# Dataset
- go to https://universe.roboflow.com/ai-camp-safety-equipment-detection/ppe-detection-using-cv/dataset/3 and click `"Download Dataset"`
- choose format `"COCO"` and `"show download code"` and `"continue"`
- choose `"Terminal"` and copy the command `"curl..."` and paste the command in the cell below.
- add `"!"` in the beginning of the command and replace `"\&gt;"` with `">"`

In [17]:
# Add below your download code from Roboflow, it should look like the following, with your unique roboflow dataset url:
# Example (with "!" added in the beginning of the command and replaced "&gt;" with ">". Also added "-q" for less output):
# !curl -L "https://universe.roboflow.com/ds/<unique-dataset-url>" > roboflow.zip; unzip -q roboflow.zip; rm roboflow.zip
!curl -L "https://universe.roboflow.com/ds/J5viLPA0jB?key=O0CBXu2KVs" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7073    0  7073    0     0  72329      0 --:--:-- --:--:-- --:--:-- 72917
Archive:  roboflow.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of roboflow.zip or
        roboflow.zip.zip, and cannot find roboflow.zip.ZIP, period.


In [18]:
# Try using wget to download the data
!wget -O roboflow.zip "https://universe.roboflow.com/ds/J5viLPA0jB?key=O0CBXu2KVs" && unzip -q roboflow.zip && rm roboflow.zip

--2025-09-24 11:31:35--  https://universe.roboflow.com/ds/J5viLPA0jB?key=O0CBXu2KVs
Resolving universe.roboflow.com (universe.roboflow.com)... 172.66.41.4, 172.66.42.252, 2606:4700:3108::ac42:2904, ...
Connecting to universe.roboflow.com (universe.roboflow.com)|172.66.41.4|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2025-09-24 11:31:35 ERROR 403: Forbidden.



In [10]:
# Move test/train/valid to dataset folder
from pathlib import Path
DATASET_PATH = 'dataset/PPE_Detection_Using_CV.v3i.coco'
if not Path(f'{DATASET_PATH}/train/_annotations.coco.json').exists():
    assert Path(f'train/_annotations.coco.json').exists()
    assert Path(f'valid/_annotations.coco.json').exists()
    assert Path(f'test/_annotations.coco.json').exists()
    !mkdir -p $DATASET_PATH
    !mv test train valid *txt $DATASET_PATH/

AssertionError: 

In [ ]:
assert Path(f'{DATASET_PATH}/train/_annotations.coco.json').exists()
assert Path(f'{DATASET_PATH}/valid/_annotations.coco.json').exists()

# Training config file: nanodet-plus-m-1.5x_416-ppe.yml
The following block of code creates the NanoDet training config file which
is based on nanodet/config/nanodet-plus-m-1.5x_416.yml.
Updated for the custom PPE dataset
Change number of `total_epochs` for better performance.

If training on more than 1 GPU, then set `gpu_ids`:
 * 2 gpu: [0,1]
 * etc...

Increase `total_epochs`, for example 20.

Feel free to increase `val_intervals`, for example 10.

## Resume training
Uncomment `resume` and `load_model` and add path to trained weights, For example: `workspace/nanodet-plus-m-1.5x_416-ppe/model_last.ckpt`

For details see NanoDet github repo and [config docs](https://github.com/RangiLyu/nanodet/blob/main/docs/config_file_detail.md). Observe recommendation to adjust `lr` with `batch_size`.

In [ ]:
%%bash
touch nanodet-plus-m-1.5x_416-ppe.yml
cat <<EOF >nanodet-plus-m-1.5x_416-ppe.yml
# Comments:
# -  based on nanodet/config/nanodet-plus-m-1.5x_416.yml
# -  "device": settings for colab T4 GPU
# -  "total_epochs": set to 20 during testing, default 300
save_dir: workspace/nanodet-plus-m-1.5x_416-ppe
model:
  weight_averager:
    name: ExpMovingAverager
    decay: 0.9998
  arch:
    name: NanoDetPlus
    detach_epoch: 10
    backbone:
      name: ShuffleNetV2
      model_size: 1.5x
      out_stages: [2,3,4]
      activation: LeakyReLU
    fpn:
      name: GhostPAN
      in_channels: [176, 352, 704]
      out_channels: 128
      kernel_size: 5
      num_extra_level: 1
      use_depthwise: True
      activation: LeakyReLU
    head:
      name: NanoDetPlusHead
      num_classes: 8
      input_channel: 128
      feat_channels: 128
      stacked_convs: 2
      kernel_size: 5
      strides: [8, 16, 32, 64]
      activation: LeakyReLU
      reg_max: 7
      norm_cfg:
        type: BN
      loss:
        loss_qfl:
          name: QualityFocalLoss
          use_sigmoid: True
          beta: 2.0
          loss_weight: 1.0
        loss_dfl:
          name: DistributionFocalLoss
          loss_weight: 0.25
        loss_bbox:
          name: GIoULoss
          loss_weight: 2.0
    # Auxiliary head, only use in training time.
    aux_head:
      name: SimpleConvHead
      num_classes: 8
      input_channel: 256
      feat_channels: 256
      stacked_convs: 4
      strides: [8, 16, 32, 64]
      activation: LeakyReLU
      reg_max: 7
data:
  train:
    name: CocoDataset
    img_path: dataset/PPE_Detection_Using_CV.v3i.coco/train
    ann_path: dataset/PPE_Detection_Using_CV.v3i.coco/train/_annotations.coco.json
    input_size: [416,416] #[w,h]
    keep_ratio: False
    pipeline:
      perspective: 0.0
      scale: [0.6, 1.4]
      stretch: [[0.8, 1.2], [0.8, 1.2]]
      rotation: 0
      shear: 0
      translate: 0.2
      flip: 0.5
      brightness: 0.2
      contrast: [0.6, 1.4]
      saturation: [0.5, 1.2]
      normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
  val:
    name: CocoDataset
    img_path: dataset/PPE_Detection_Using_CV.v3i.coco/valid
    ann_path: dataset/PPE_Detection_Using_CV.v3i.coco/valid/_annotations.coco.json
    input_size: [416,416] #[w,h]
    keep_ratio: False
    pipeline:
      normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
device:
  gpu_ids: [0]
  workers_per_gpu: 2
  batchsize_per_gpu: 32
  precision: 32 # set to 16 to use AMP training
schedule:
#  resume:
#  load_model:
  optimizer:
    name: AdamW
    lr: 0.001
    weight_decay: 0.05
  warmup:
    name: linear
    steps: 500
    ratio: 0.0001
  total_epochs: 5
  lr_schedule:
    name: CosineAnnealingLR
    T_max: 300
    eta_min: 0.00005
  val_intervals: 5
grad_clip: 35
evaluator:
  name: CocoDetectionEvaluator
  save_key: mAP
log:
  interval: 10

class_names: [
  'safety-equipment',
  'person',
  'goggles',
  'helmet',
  'no-goggles',
  'no-helmet',
  'no-vest',
  'vest']
EOF

# Training

In [ ]:
# OBSERVE: update the following assert statement to match your yml file settings.

import yaml
with open('nanodet-plus-m-1.5x_416-ppe.yml', 'r') as file:
    config = yaml.safe_load(file)

gpu_ids = config['device']['gpu_ids']
assert isinstance(gpu_ids, list) and gpu_ids, print(f"gpu_ids: {config['device']['gpu_ids']}")
assert config['schedule']['total_epochs'] == 5 or config['schedule']['total_epochs'] == 20, print(f"total_epochs: {config['schedule']['total_epochs']}")
assert config['schedule']['val_intervals'] == 5 or config['schedule']['val_intervals'] == 10, print(f"val_intervals: {config['schedule']['val_intervals']}")

In [ ]:
import torch
assert '2.5.' in torch.__version__, print(torch.__version__)
assert Path('nanodet-plus-m-1.5x_416-ppe.yml').exists()
!export PYTHONPATH=$PWD/nanodet:$PYTHONPATH && python nanodet/tools/train.py nanodet-plus-m-1.5x_416-ppe.yml

# Remove aux layers that are only used during training

In [ ]:
import sys
sys.path.insert(0,"./nanodet")

import copy
import torch
from nanodet.model.arch import build_model
from nanodet.util import cfg, load_config, Logger

def remove_aux(cfg, model_path, remove_layers=['aux_fpn', 'aux_head'], debug=False):
    model = build_model(cfg.model)
    ckpt = torch.load(model_path, map_location=lambda storage, loc: storage)
    if len(remove_layers) > 0:
        state_dict = copy.deepcopy(ckpt['state_dict'])
        for rlayer in remove_layers:
            for layer in ckpt['state_dict']:
                if rlayer in layer:
                    del state_dict[layer]
                    if debug:
                        print(f'removed layer: {layer}')
        del ckpt['state_dict']
        ckpt['state_dict'] = copy.deepcopy(state_dict)
        del state_dict
    return ckpt

In [ ]:
config_path = 'nanodet-plus-m-1.5x_416-ppe.yml'
model_path = 'workspace/nanodet-plus-m-1.5x_416-ppe/model_best/nanodet_model_best.pth'
dst_path = 'workspace/nanodet-plus-m-1.5x_416-ppe/model_best/nanodet_model_best-removed-aux.pth'

load_config(cfg, config_path)
ckpt = remove_aux(cfg, model_path, ['aux_fpn', 'aux_head'])
torch.save(ckpt, dst_path)
print(f'Saved to: {dst_path}')

In [ ]:
# Compare size w and w/o aux
!ls -l workspace/nanodet-plus-m-1.5x_416-ppe/model_best

# Quantization of custom NanoDet model using Model Compression Toolkit
Quantization is based on examples from [Model Compression Toolkit Keras Tutorials](https://github.com/sony/model_optimization/blob/v2.0.0/tutorials/notebooks/keras/ptq)

# Installation

In [ ]:
import sys
import importlib

if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install -q model_compression_toolkit==2.3.0

# Observe, we use tutorials assets from mct version 2.2.0
!rm -rf temp_mct
!git clone https://github.com/sony/model_optimization.git temp_mct && cd temp_mct && git checkout v2.2.0 && cd ..
!mv temp_mct/tutorials . && rm -rf temp_mct
sys.path.insert(0,"tutorials")

# Keras NanoDet float model

In [ ]:
from pathlib import Path

import numpy as np
import tensorflow as tf
import torch
assert '2.15.' in tf.__version__, print(tf.__version__)
assert '2.5.' in torch.__version__, print(torch.__version__)

from keras.models import Model
import model_compression_toolkit as mct
from tutorials.mct_model_garden.models_keras.nanodet.nanodet_keras_model import nanodet_plus_m
from tutorials.mct_model_garden.models_keras.utils.torch2keras_weights_translation import load_state_dict
from tutorials.mct_model_garden.models_keras.nanodet.nanodet_keras_model import nanodet_box_decoding

In [ ]:
# Upload the trained custom model
CUSTOM_WEIGHTS_FILE = dst_path  # The NanoDet model trained with PPE dataset
CLASS_NAMES = [
  'safety-equipment',
  'person',
  'goggles',
  'helmet',
  'no-goggles',
  'no-helmet',
  'no-vest',
  'vest']
NUM_CLASSES = len(CLASS_NAMES)

DATASET_TRAIN = 'dataset/PPE_Detection_Using_CV.v3i.coco/train'
ANNOT_TRAIN = 'dataset/PPE_Detection_Using_CV.v3i.coco/train/_annotations.coco.json'
DATASET_VALID = 'dataset/PPE_Detection_Using_CV.v3i.coco/valid'
ANNOT_VALID = 'dataset/PPE_Detection_Using_CV.v3i.coco/valid/_annotations.coco.json'
DATASET_REPR = DATASET_VALID
ANNOT_REPR = ANNOT_VALID

QUANTIZED_MODEL = 'nanodet-quant-ppe.keras'

BATCH_SIZE = 5
N_ITER = 20  # 1 for testing, otherwise 20

assert Path(CUSTOM_WEIGHTS_FILE).exists()
assert Path(DATASET_REPR).exists()

In [ ]:
def get_model(weights=CUSTOM_WEIGHTS_FILE, num_classes=NUM_CLASSES):
    INPUT_RESOLUTION = 416
    INPUT_SHAPE = (INPUT_RESOLUTION, INPUT_RESOLUTION, 3)
    SCALE_FACTOR = 1.5
    BOTTLENECK_RATIO = 0.5
    FEATURE_CHANNELS = 128

    pretrained_weights = torch.load(weights, map_location=torch.device('cpu'))['state_dict']
    # Generate Nanodet base model
    model = nanodet_plus_m(INPUT_SHAPE, SCALE_FACTOR, BOTTLENECK_RATIO, FEATURE_CHANNELS, num_classes)

    # Set the pre-trained weights
    load_state_dict(model, state_dict_torch=pretrained_weights)

    # Add Nanodet Box decoding layer (decode the model outputs to bounding box coordinates)
    scores, boxes = nanodet_box_decoding(model.output, res=INPUT_RESOLUTION, num_classes=num_classes)

    # Add TensorFlow NMS layer
    outputs = tf.image.combined_non_max_suppression(
        boxes,
        scores,
        max_output_size_per_class=300,
        max_total_size=300,
        iou_threshold=0.65,
        score_threshold=0.001,
        pad_per_class=False,
        clip_boxes=False
        )

    model = Model(model.input, outputs, name='Nanodet_plus_m_1.5x_416')

    print('Model is ready for evaluation')
    return model

In [ ]:
# known warning:  WARNING: head.distribution_project.project not assigned to keras model !!!
float_model = get_model(CUSTOM_WEIGHTS_FILE, NUM_CLASSES)

# PTQ quantization

In [ ]:
from typing import Callable, Iterator, Tuple, List

import cv2
from tutorials.mct_model_garden.evaluation_metrics.coco_evaluation import coco_dataset_generator, CocoEval
from edgemdt_tpc import get_target_platform_capabilities

def nanodet_preprocess(x):
    img_mean = [103.53, 116.28, 123.675]
    img_std = [57.375, 57.12, 58.395]
    x = cv2.resize(x, (416, 416))
    x = (x - img_mean) / img_std
    return x

def get_representative_dataset(n_iter: int, dataset_loader: Iterator[Tuple]):
    def representative_dataset() -> Iterator[List]:
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

def quantization(float_model, dataset, annot, n_iter=N_ITER):
    # Load representative dataset
    representative_dataset = coco_dataset_generator(dataset_folder=dataset,
                                                    annotation_file=annot,
                                                    preprocess=nanodet_preprocess,
                                                    batch_size=BATCH_SIZE)

    # Get a TPC object representing the imx500 hardware
    tpc = get_target_platform_capabilities(tpc_version='4.0', device_type='imx500')

    # Preform post training quantization
    quant_model, _ = mct.ptq.keras_post_training_quantization(
        float_model,
        representative_data_gen=get_representative_dataset(n_iter, representative_dataset),
        target_platform_capabilities=tpc)

    print('Quantized model is ready')
    return quant_model

In [ ]:
quant_model = quantization(float_model, DATASET_REPR, ANNOT_REPR)
print(f'Representative dataset: {DATASET_REPR}')

In [ ]:
# Observe that loading quantized model might require specification of custom layers,
# see https://github.com/sony/model_optimization/issues/1104
mct.exporter.keras_export_model(model=quant_model, save_model_path=QUANTIZED_MODEL)
print(f'Quantized model saved: {QUANTIZED_MODEL}')

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tutorials.mct_model_garden.evaluation_metrics.coco_evaluation import CocoDataset, model_predict, load_and_preprocess_image
from tutorials.mct_model_garden.models_pytorch.yolov8.yolov8_postprocess import clip_boxes, clip_coords, scale_coords, scale_boxes

In [ ]:
"""
Helper function to perform coco evaluation with custom dataset
"""
from typing import List, Dict, Tuple, Callable
import numpy as np

# slow no batch version
def format_results(outputs: List, img_ids: List, orig_img_dims: List, output_resize: Dict, custom_labels: Callable) -> List[Dict]:
    """
    Format model outputs into a list of detection dictionaries.

    Args:
        outputs (list): List of model outputs, typically containing bounding boxes, scores, and labels.
        img_ids (list): List of image IDs corresponding to each output.
        orig_img_dims (list): List of tuples representing the original image dimensions (h, w) for each output.
        output_resize (Dict): Contains the resize information to map between the model's
                 output and the original image dimensions.
        custom_labels (Callable): A function to map label outputs. Typically, COCO re-map from 80 (model) to 91 (dataset)

    Returns:
        list: A list of detection dictionaries, each containing information about the detected object.
    """
    detections = []
    h_model, w_model = output_resize['shape']
    preserve_aspect_ratio = output_resize['aspect_ratio_preservation']

    image_id = img_ids
    scores = outputs[1].numpy().squeeze()  # Extract scores
    labels = (custom_labels(outputs[2].numpy())).squeeze()  # Provide a function to map label outputs
    boxes = outputs[0].numpy().squeeze()  # Extract bounding boxes
    boxes = scale_boxes(boxes, orig_img_dims[0], orig_img_dims[1], h_model, w_model, preserve_aspect_ratio)
    for score, label, box in zip(scores, labels, boxes):
        if score == 0.0:
            continue
        detection = {
            "image_id": image_id,
            "category_id": label,
            "bbox": [box[1], box[0], box[3] - box[1], box[2] - box[0]],
            "score": score
        }
        detections.append(detection)
    return detections

In [ ]:
custom_dataset = CocoDataset(dataset_folder=DATASET_VALID,
                             annotation_file=ANNOT_VALID,
                             preprocess=nanodet_preprocess)

MODEL = float_model
INPUT_RESOLUTION = 416

output_resize = {'shape': (INPUT_RESOLUTION, INPUT_RESOLUTION), 'aspect_ratio_preservation': False}
coco_predictions = []
for idx, (im, anns) in enumerate(custom_dataset):
    #if idx > 2:
    #    continue
    outputs = model_predict(MODEL, np.expand_dims(im, axis=0))  # 4 tensors: bbox, scores, classes, detections
    detections = format_results(outputs, anns[0]['image_id'], anns[0]['orig_img_dims'], output_resize, lambda x: x)
    coco_predictions.extend(detections)
    if (idx + 1) % 50 == 0:
        print(f'processed {(idx + 1)} images')

print(len(coco_predictions))

cocoGt=COCO(ANNOT_VALID)
cocoDt=cocoGt.loadRes(coco_predictions)
cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()
"""
epochs = 20
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.507
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.214
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.304
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.242
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.439
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.484
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.228
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.370
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.560
"""

In [ ]:
custom_dataset = CocoDataset(dataset_folder=DATASET_VALID,
                             annotation_file=ANNOT_VALID,
                             preprocess=nanodet_preprocess)

MODEL = quant_model
INPUT_RESOLUTION = 416

output_resize = {'shape': (INPUT_RESOLUTION, INPUT_RESOLUTION), 'aspect_ratio_preservation': False}
coco_predictions = []
for idx, (im, anns) in enumerate(custom_dataset):
    #if idx > 2:
    #    continue
    outputs = model_predict(MODEL, np.expand_dims(im, axis=0))  # 4 tensors: bbox, scores, classes, detections
    detections = format_results(outputs, anns[0]['image_id'], anns[0]['orig_img_dims'], output_resize, lambda x: x)
    coco_predictions.extend(detections)
    if (idx + 1) % 50 == 0:
        print(f'processed {(idx + 1)} images')

print(len(coco_predictions))

cocoGt=COCO(ANNOT_VALID)
cocoDt=cocoGt.loadRes(coco_predictions)
cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()
"""
epochs = 20
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.242
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.495
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.206
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.295
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.237
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.427
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.361
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.553
"""

# Visualize detection

In [ ]:
# Helper functions for visualization
import numpy as np

# draw a single bounding box onto a numpy array image
def draw_bounding_box(img, annotation, scale, class_id, score):
    row = scale[0]
    col = scale[1]
    x_min, y_min = int(annotation[1]*col), int(annotation[0]*row)
    x_max, y_max = int(annotation[3]*col), int(annotation[2]*row)

    color = (0,255,0)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)
    text = f'{int(class_id)}: {score:.2f}'
    cv2.putText(img, text, (x_min + 10, y_min + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# draw all annotation bounding boxes on an image
def annotate_image(img, output, scale, threshold=0.55):
    b = output[0].numpy()[0]
    s = output[1].numpy()[0]
    c = output[2].numpy()[0]
    for index, row in enumerate(b):
        if s[index] > threshold:
            #print(f'row: {row}')
            id = int(c[index])
            draw_bounding_box(img, row, scale, id, s[index])
            print(f'class: {CLASS_NAMES[id]} ({id}), score: {s[index]:.2f}')
    return {'bbox':b, 'score':s, 'classes':c}

In [ ]:
# See appendix for results. For 2 epochs, the bounding boxes are not perfect...
# But improves considerably for 20 epochs.

MODEL = quant_model

test_img = 'dataset/PPE_Detection_Using_CV.v3i.coco/valid/image_257_jpg.rf.1a3a6eb456134cce302712c109645c26.jpg'
img = load_and_preprocess_image(f'{test_img}', nanodet_preprocess)
output = MODEL(np.expand_dims(img, axis=0))
image = cv2.imread(f'{test_img}')
print(f'image shape: {image.shape}')
r = annotate_image(image, output, scale=image.shape)
assert r['score'][0] > 0.5, print(f"r['score'][0] > 0.5 failed: {r['score'][0]}")
dst = f'annotated.jpg'
if cv2.imwrite(dst, image):
    print(f'Annotated image saved to: {dst}')
else:
    print(f'Failed saving annotated image')
from matplotlib import pyplot as plt
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# Conversion
For details see
* [Raspberry Pi Documentation](https://www.raspberrypi.com/documentation/accessories/ai-camera.html#conversion)
* [Sony IMX500 Converter documentation](https://developer.aitrios.sony-semicon.com/en/raspberrypi-ai-camera/documentation/imx500-converter)

In [ ]:
!imxconv-tf -i {QUANTIZED_MODEL} -o converted

In [ ]:
"""
# Expected output from converter:
dnnParams.xml		 nanodet-quant-ppe_MemoryReport.json
nanodet-quant-ppe.pbtxt  packerOut.zip
"""
!ls converted
assert os.path.exists("converted/packerOut.zip"), f"Converted file not found"

# Next step
__OBSERVE__: First, save the quantized model and the output from the conversion to your local machine. For packaging you will need the `packerOut.zip` file.

Next step is to package the model for IMX500, see [Raspberry Pi Documentation](https://www.raspberrypi.com/documentation/accessories/ai-camera.html#packaging)